In [ ]:
import os, json
from tools import ONEDRIVE_PATH

file_path = os.path.join(ONEDRIVE_PATH, 'data', 'FINAL', '2C', 'Lv3_4', 'SS0163_workbook', 'SS0163.json')

with open(file_path, 'r', encoding='utf-8') as f:
    origin = json.load(f)

In [ ]:
# page가 105일 때 index 찾기
target_page = "0375"  # page는 문자열로 "0105" 형식으로 저장되어 있음

for idx, content in enumerate(origin['contents']):
    if content.get('page') == target_page:
        print(f"Index: {idx}")
        print(f"Page: {content.get('page')}")
        print(f"Chapter: {content.get('chapter')}")
        break
else:
    print(f"Page {target_page} not found")

In [ ]:
import re

# 1. '문제로 확인하기' 이후 텍스트 추출
start = origin['contents'][idx]['page_contents'].find("문제로 확인하기")
if start == -1:
    print("'문제로 확인하기'를 찾을 수 없습니다.")
else:
    end = start + len("문제로 확인하기")
    target_text = origin['contents'][idx]['page_contents'][end:]
    original_page_contents = origin['contents'][idx]['page_contents']
    
    # 2. 패턴: 01 질문\nSolution\n답변\n02 질문\nSolution\n답변 ...
    # 질문 부분: 여러 줄 가능 (태그 포함 가능), \nSolution 전까지 (lookahead 사용)
    # 답변 부분: Solution 다음부터 첫 번째 \n까지만 (한 줄만)
    pattern = r'(?:^|\n)([0-9]+)\s+(.*?)(?=\nSolution)\nSolution\n([^\n]*)'
    
    # 3. 모든 문제 찾기 (전체 텍스트에서 매칭 위치도 함께 찾기)
    matches_with_pos = []
    for match in re.finditer(pattern, target_text, re.DOTALL):
        matches_with_pos.append({
            'match': match,
            'groups': match.groups(),
            'start': match.start(),
            'end': match.end()
        })
    
    print(f"찾은 문제 개수: {len(matches_with_pos)}")
    
    # 4. 각 문제를 number, question, answer로 분리하여 저장하고 본문 대치
    # 역순으로 처리하여 인덱스가 꼬이지 않도록 함
    page_contents = original_page_contents
    replacement_positions = []  # (start_pos, end_pos, tag) 저장
    
    for i, match_info in enumerate(matches_with_pos):
        match = match_info['match']
        number = match.group(1).strip()
        question = match.group(2).strip()
        answer = match.group(3).strip()
        
        # 태그 생성
        tag = f"q_{origin['contents'][idx]['page']:04}_{i+1:04}"
        
        print(f"번호: {number}, 태그: {tag}")
        
        # JSON 구조에 맞게 추가
        q_format = {
            "tag": tag,
            "type": "question",
            "description": { 
                "number": number, 
                "question": question.replace("\\'", "'"), 
                "options": None,
                "answer": answer, 
                "explanation": ""
            }, 
            "caption": [], 
            "file_path": None, 
            "bbox": None
        }
        
        origin['contents'][idx]['add_info'].append(q_format)
        
        # 본문에서 대치할 위치 저장 (전체 page_contents 기준)
        # '문제로 확인하기' 이후의 위치를 전체 page_contents 기준으로 변환
        full_start = end + match.start()
        full_end = end + match.end()
        replacement_positions.append((full_start, full_end, tag))
    
    # 역순으로 대치 (뒤에서부터 처리하여 인덱스가 꼬이지 않도록)
    replacement_positions.sort(reverse=True, key=lambda x: x[0])
    
    for full_start, full_end, tag in replacement_positions:
        page_contents = (
            page_contents[:full_start] + 
            f"\n{{{tag}}}" + 
            page_contents[full_end:]
        )
    
    # 본문 업데이트
    origin['contents'][idx]['page_contents'] = page_contents
    print(f"\n본문 대치 완료: {len(replacement_positions)}개 문제를 태그로 대치했습니다.")

In [ ]:
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(origin, f, ensure_ascii=False, indent=4)

In [ ]:
from tools.data_processing.json_cleaner import JSONCleaner

cleaner = JSONCleaner()
cleaner.cleanup_file(file_path, create_backup=False)